# Model Validation with TensorFlow Model Analysis

In this lab, we use [TensorFlow Model Analysis](https://www.tensorflow.org/tfx/guide/tfma) to assess the quality of the trained model. This lab covers the following:
1. **Export** evaluation saved model
2. Define **data slices** for analysis
3. Generat **evaluation** the metrics
4. **Visualize** results

In [ ]:
#!pip install -q tensorflow-model-analysis

In [1]:
import os
import tensorflow as tf
import tensorflow.io as tf_io
import tensorflow_transform as tft
import tensorflow_model_analysis as tfma
import tensorflow_data_validation as tfdv
from tensorflow_transform.tf_metadata import schema_utils

In [2]:
WORKSPACE = 'workspace' # you can set to a GCS location
RAW_SCHEMA_LOCATION = os.path.join(WORKSPACE, 'raw_schema.pbtxt')
DATA_DIR = os.path.join(WORKSPACE, 'raw_data')
TRANSFORM_ARTEFACTS_DIR = os.path.join(WORKSPACE, 'transform_artifacts')
DATA_FILES = os.path.join(DATA_DIR,'*.csv')
MODELS_DIR = os.path.join(WORKSPACE, 'models')
MODEL_NAME = 'dnn_classifier'
MODEL_DIR = os.path.join(MODELS_DIR, MODEL_NAME)

### Load TFT Outputs

In [11]:
transform_output = tft.TFTransformOutput(TRANSFORM_ARTEFACTS_DIR)

## 1. Export Evaluation Saved Model

In [5]:
HEADER = ['age', 'workclass', 'fnlwgt', 'education', 'education_num',
          'marital_status', 'occupation', 'relationship', 'race', 'gender',
          'capital_gain', 'capital_loss', 'hours_per_week',
          'native_country', 'income_bracket']

HEADER_DEFAULTS = [[0], [''], [0], [''], [0], [''], [''], [''], [''], [''],
                   [0], [0], [0], [''], ['']]

TARGET_FEATURE_NAME = 'income_bracket'
TARGET_LABELS = [' <=50K', ' >50K']
WEIGHT_COLUMN_NAME = 'fnlwgt'

### 1.1 Implement eval_input_receiver_fn
This function expect **raw** data interface, then it applies the **transformation**

In [12]:
def eval_input_receiver_fn():
    
    receiver_tensors = {'examples': tf.placeholder(dtype=tf.string, shape=[None])}
    columns = tf.decode_csv(receiver_tensors['examples'], record_defaults=HEADER_DEFAULTS)
    
    features = dict(zip(HEADER, columns))
    
    for feature_name in features:
        if features[feature_name].dtype == tf.int32:
            features[feature_name] = tf.cast(features[feature_name], tf.int64)
        features[feature_name] = tf.reshape(features[feature_name], (-1, 1))
        
    transformed_features = transform_output.transform_raw_features(features)
    features.update(transformed_features)

    return tfma.export.EvalInputReceiver(
        features=features,
        receiver_tensors=receiver_tensors,
        labels=features[TARGET_FEATURE_NAME]
    )

### 1.2 Export an evaluation saved model
First, we load the estimator...

In [13]:
import joblib
class Parameters: pass
estimator = joblib.load( './estimator.joblib')

In [14]:
def update_optimizer(initial_learning_rate, decay_steps):
    learning_rate = tf.train.cosine_decay_restarts(
        initial_learning_rate,
        tf.train.get_global_step(),
        first_decay_steps=50,
        t_mul=2.0,
        m_mul=1.0,
        alpha=0.0,
    )
    
    tf.summary.scalar('learning_rate', learning_rate)
    return tf.train.AdamOptimizer(learning_rate=learning_rate)

def metric_fn(labels, predictions):
    
    metrics = {}
    label_index = tf.contrib.lookup.index_table_from_tensor(tf.constant(TARGET_LABELS)).lookup(labels)
    one_hot_labels = tf.one_hot(label_index, len(TARGET_LABELS))
    
    metrics['mirco_accuracy'] = tf.metrics.mean_per_class_accuracy(
        labels=label_index,
        predictions=predictions['class_ids'],
        num_classes=2
    )
    
    return metrics

In [15]:
TARGET_FEATURE_NAME = 'income_bracket'
TARGET_LABELS = [' <=50K', ' >50K']
WEIGHT_COLUMN_NAME = 'fnlwgt'

In [16]:
tf.logging.set_verbosity(tf.logging.ERROR)

eval_model_dir = os.path.join(MODEL_DIR, "export/evaluate")
if tf_io.gfile.exists(eval_model_dir):
    tf_io.gfile.rmtree(eval_model_dir)

eval_model_dir = tfma.export.export_eval_savedmodel(
        estimator=estimator,
        export_dir_base=eval_model_dir,
        eval_input_receiver_fn=eval_input_receiver_fn
)

eval_model_dir

b'workspace/models/dnn_classifier/export/evaluate/1581343523'

In [18]:
#!saved_model_cli show --dir=${eval_model_dir} --all

## 2. Define Slices for Evaluation

In [19]:
slice_spec = [
  tfma.slicer.SingleSliceSpec(),
  tfma.slicer.SingleSliceSpec(columns=['occupation'])
]

## 3. Generate evaluation metrics

You can run this on Dataflow by setting the `pipeline_options` parameter.

In [24]:
eval_result = tfma.run_model_analysis(
    eval_shared_model=tfma.default_eval_shared_model(
        eval_saved_model_path=eval_model_dir,
        example_weight_key=WEIGHT_COLUMN_NAME) , 
    data_location=DATA_FILES, 
    file_format='text', 
    slice_spec=slice_spec,  
    output_path=None
)

In [27]:
eval_result.slicing_metrics

[((('occupation', ' Handlers-cleaners'),),
  {'': {'': {'accuracy': {'doubleValue': 0.9415510296821594},
     'accuracy_baseline': {'doubleValue': 0.9357296824455261},
     'auc': {'doubleValue': 0.9308984875679016},
     'auc_precision_recall': {'doubleValue': 0.47234880924224854},
     'average_loss': {'doubleValue': 0.14452044665813446},
     'label/mean': {'doubleValue': 0.06427029520273209},
     'mirco_accuracy': {'doubleValue': 0.553313672542572},
     'post_export_metrics/example_count': {'doubleValue': 2072.0},
     'post_export_metrics/example_weight': {'doubleValue': 418652869.0},
     'precision': {'doubleValue': 0.8398988842964172},
     'prediction/mean': {'doubleValue': 0.06253305077552795},
     'recall': {'doubleValue': 0.11190605163574219}}}}),
 ((('occupation', ' Priv-house-serv'),),
  {'': {'': {'accuracy': {'doubleValue': 0.9944964647293091},
     'accuracy_baseline': {'doubleValue': 0.9924167394638062},
     'auc': {'doubleValue': 0.9962059259414673},
     'auc_pr

## 4. Visalise and analyze evalation results

In [26]:
tfma.view.render_slicing_metrics(
    result=eval_result, 
    slicing_column='occupation'
)

SlicingMetricsViewer(config={'weightedExamplesColumn': 'post_export_metrics/example_weight'}, data=[{'metrics'…